In [1]:
import numpy as np
import operator

from typing import NamedTuple
from pathlib import Path
from uuid import uuid4
from shutil import copytree
from pintFoam.utils import pushd

from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile
from PyFoam.RunDictionary.SolutionDirectory import SolutionDirectory
from PyFoam.Execution.UtilityRunner import UtilityRunner

In [5]:
def solution_directory(case):
    return SolutionDirectory(case.path)


def parameter_file(case, relative_path):
    return ParsedParameterFile(case.path / relative_path)


def time_directory(case):
    return solution_directory(case)[case.time]


class BaseCase(NamedTuple):
    """Base case is a cleaned version of the system. If it contains any fields,
    it will only be the `0` time. Any field that is copied for manipulation will
    do so on top of an available base case in the `0` slot."""
    root: Path
    case: str
    
    @property
    def path(self):
        return self.root / self.case
    
    def new_vector(self, name=None):
        """Creates new `Vector` using this base case."""
        new_case = name or uuid4().hex
        new_path = self.root / new_case
        if not new_path.exists():
            copytree(self.path, new_path)
        return Vector(self, new_case, 0)


class Vector(NamedTuple):
    base: BaseCase
    case: str
    time: int
    
    @property
    def path(self):
        return self.base.root / self.case
    
    @property
    def files(self):
        return time_directory(self).getFiles()    

    def clone(self):
        x = self.base.new_vector()
        for f in self.files:
            r_f = self.internalField(f)
            x_content = time_directory(x)[f].getContent()
            x_content.content['internalField'].val[:] = r_f
            x_content.writeFile()
        return x
            
    def internalField(self, key):
        return np.array(time_directory(self)[key].getContent().content['internalField'])
    
    def _operate_vec_vec(self, other: Vector, op):
        x = self.base.new_vector()
        for f in self.files:
            a_f = self.internalField(f)
            b_f = other.internalField(f)
            x_content = time_directory(x)[f].getContent()
            x_f = x_content.content['internalField'].val[:] = op(a_f, b_f)
            x_content.writeFile()
        return x
    
    def _operate_vec_scalar(self, s: float, op):
        x = self.base.new_vector()
        for f in self.files:
            a_f = self.internalField(f)
            x_content = time_directory(x)[f].getContent()
            x_f = x_content.content['internalField'].val[:] = op(a_f, s)
            x_content.writeFile()
        return x        
        
    def __sub__(self, other: Vector):
        return self._operate_vec_vec(other, operator.sub)
    
    def __add__(self, other: Vector):
        return self._operate_vec_vec(other, operator.add)
    
    def __mul__(self, scale: float):
        return self._operate_vec_scalar(scale, operator.mul)


def setFields(v, *, defaultFieldValues, regions):
    x = parameter_file(v, "system/setFieldsDict")
    x['defaultFieldValues'] = defaultFieldValues
    x['regions'] = regions
    x.writeFile()
    
    with pushd(v.path):
        u = UtilityRunner(argv=['setFields'], silent=True)
        u.start()

In [6]:
blob_base = BaseCase(Path("./blob"), "baseCase")
a = blob_base.new_vector("a")
b = blob_base.new_vector("b")

In [7]:
setFields(
    a, defaultFieldValues=['volScalarFieldValue', 'T', 260],
    regions=['sphereToCell', {'centre': [4, 5, 0],
                              'radius': 2.0,
                              'fieldValues': ['volScalarFieldValue', 'T', 300]}])

setFields(
    b, defaultFieldValues=['volScalarFieldValue', 'T', 260],
    regions=['sphereToCell', {'centre': [6, 5, 0],
                              'radius': 2.0,
                              'fieldValues': ['volScalarFieldValue', 'T', 300]}])

In [8]:
c = a - b

In [9]:
d = a + b

In [10]:
e = a * 10.0

In [11]:
x = time_directory(e)['T'].getContent()

In [12]:
q = x.content['internalField']

In [13]:
np.array(q)

array([2600., 2600., 2600., ..., 2600., 2600., 2600.])

In [91]:
a = np.array(q.val)

In [92]:
q.val[:] = a*2

'nonuniform List<scalar> 2500
(
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  600
  